In [ ]:
!pip install transformers

In [ ]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install unsloth

In [ ]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')
from huggingface_hub import notebook_login
HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import unsloth
from unsloth import FastLanguageModel

In [ ]:
max_seq_length = 1024
dtype = None
load_in_4bits = True
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bits,
    token = HF_TOKEN
    )


In [ ]:
model = FastLanguageModel.get_peft_model(model,
                                    r=16,
                                    target_modules = [
                                        "q_proj",
                                        "k_proj",
                                        "v_proj",
                                        "o_proj",
                                        "gate_proj",
                                        "up_proj",
                                        "down_proj",
                                    ],
                                    lora_alpha = 16,
                                    lora_dropout = 0.05,
                                    bias = "none",
                                    use_gradient_checkpointing = "unsloth",
                                    random_state=3407,
                                    use_rslora=False,
                                    loftq_config=None
                                    )

In [ ]:
prompt_style = """below is an instruction that describes a task ,paored with an input that provides further context.write a response that appropriately completes the request.before answering ,think carefully about the question and create a step-by-step cahin of thoughts to ensurea logical and accurate response.

##instruction:
you are an acadamice chatbot for zarqa university and your name is chat zu from now on you can anwser any thing about the university but always remind the user that your a zarqa chat bot when ever they ask for anything not about the university allways trie to get students and there perints to join the university try to be as helpfull, use the zarqa university to answer relevent questions,i add a algorithem to predict what courses should the student takeand to find out if the lecyurers are avalable.

##Topic:
{}

##Response:
<think>
{}
</think>
{}"""

In [ ]:
import zipfile

zip_path = "/content/Data.zip"

# Changed extract_dir to be a directory
extract_dir = "/content/"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")

In [ ]:
import os
import json

def txt_folder_to_proper_json(folder_path, output_json_path):
    """
    Convert all txt files in a folder to proper JSON format with combined objects.

    Args:
        folder_path (str): Path to the folder containing txt files
        output_json_path (str): Path where to save the output JSON file
    """
    data = []

    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)

            # Read the file content
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

            # Create single object with both properties
            file_data = {
                "name": filename,
                "text": content
            }
            data.append(file_data)

    # Write to JSON file
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

# Example usage:
txt_folder_path = '/content/Data'
output_json_path = '/content/output.json'
txt_folder_to_proper_json(txt_folder_path, output_json_path)

In [ ]:
with open('output.json', 'r') as f:
    files_data = json.load(f)

for file_obj in files_data:
    print(f"Filename: {file_obj['name']}")
    print(f"Content: {file_obj['text'][:100]}...")  # First 100 chars
    print("-" * 40)

In [ ]:
files_data

In [ ]:
import json

def load_json_file(json_path):
    """
    Load the JSON file and return its contents

    Args:
        json_path (str): Path to the JSON file

    Returns:
        The parsed JSON data
    """
    with open(json_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
    return data


data = load_json_file('/content/output.json')

In [ ]:
data

In [ ]:
for i in range(0, len(data), 2):
    name_entry = data[i]
    text_entry = data[i]

    filename = name_entry['name']
    content = text_entry['text']

    print(f"File: {filename}")
    print(f"Content: {content[:50]}...")  # Print first 50 chars
    print("---")

In [ ]:
#EOS = tokenizer.eos_token
def formating_prompt_func(examples):
    name = examples.get("name")  # Use get to handle missing keys
    text = examples.get("text")  # Use get to handle missing keys

    if name is not None and text is not None:  # Check if both keys are present
        text = prompt_style.format(name, text, '')
        # + EOS
        return {"text": text}  # Return as a list for consistency
    else:
        return {"text": ""}  # Return empty list if either key is missing

In [ ]:
!pip install datasets

In [ ]:
from collections import deque

class Course:
    def __init__(self, name, credits, sections, category, prerequisites=[]):
        self.name = name
        self.credits = credits
        self.sections = sections  # List of possible time slots
        self.category = category  # e.g., 'core', 'elective', 'lab'
        self.prerequisites = prerequisites

def topological_sort(courses):
    in_degree = {course: 0 for course in courses}
    for course in courses:
        for prereq in course.prerequisites:
            in_degree[course] += 1

    queue = deque([course for course in courses if in_degree[course] == 0])
    sorted_courses = []

    while queue:
        course = queue.popleft()
        sorted_courses.append(course)
        for dependent in courses:
            if course in dependent.prerequisites:
                in_degree[dependent] -= 1
                if in_degree[dependent] == 0:
                    queue.append(dependent)

    return sorted_courses

def schedule_courses(courses, max_credits=18, taken_courses=set(), category_requirements={}):
    sorted_courses = topological_sort(courses)
    schedule = []
    completed_courses = set(taken_courses)
    category_credits_taken = {cat: 0 for cat in category_requirements}

    while sorted_courses:
        current_semester = []
        current_credits = 0
        occupied_time_slots = set()
        remaining_courses = []

        for course in sorted_courses:
            if course.name in completed_courses:
                continue

            if any(prereq.name not in completed_courses for prereq in course.prerequisites):
                remaining_courses.append(course)
                continue

            # Skip if category requirement is already fulfilled
            if course.category in category_requirements and category_credits_taken[course.category] >= category_requirements[course.category]:
                continue

            # Choose an available section
            chosen_section = None
            for section in course.sections:
                if section not in occupied_time_slots:
                    chosen_section = section
                    break

            if chosen_section and current_credits + course.credits <= max_credits:
                current_semester.append((course, chosen_section))
                current_credits += course.credits
                occupied_time_slots.add(chosen_section)
                category_credits_taken[course.category] += course.credits
            else:
                remaining_courses.append(course)

        if current_semester:
            schedule.append(current_semester)
            completed_courses.update(c.name for c, _ in current_semester)

        sorted_courses = remaining_courses

    return schedule

In [ ]:
# Example Usage

def make_schedule():
    c1 = Course("Math 101", 3, [('Mon', '10:00-12:00'), ('Wed', '14:00-16:00')], 'core')
    c2 = Course("CS 101", 4, [('Tue', '14:00-16:00'), ('Thu', '10:00-12:00')], 'core', [c1])
    c3 = Course("Physics 101", 4, [('Mon', '12:00-14:00'), ('Fri', '16:00-18:00')], 'core')
    c4 = Course("CS 102", 4, [('Wed', '10:00-12:00'), ('Thu', '14:00-16:00')], 'core', [c2])
    c5 = Course("Elective 1", 3, [('Thu', '16:00-18:00'), ('Fri', '14:00-16:00')], 'elective')
    c6 = Course("Elective 2", 4, [('Fri', '10:00-12:00'), ('Mon', '08:00-10:00')], 'elective')
    c7 = Course("Elective 3", 3, [('Tue', '16:00-18:00'), ('Wed', '08:00-10:00')], 'elective')

    courses = [c1, c2, c3, c4, c5, c6, c7]
    taken_courses = {"Math 101"}
    category_requirements = {'core': 12, 'elective': 6}  # Example: 12 credits core, 6 credits elective

    schedule = schedule_courses(courses, taken_courses=taken_courses, category_requirements=category_requirements)

    for i, semester in enumerate(schedule):
        print(f"Semester {i+1}:")
        total_credits = sum(course.credits for course, _ in semester)
        print(f"  Total Credits: {total_credits}")
        for course, section in semester:
            print(f"  {course.name} ({course.credits} credits, {course.category}) - {section}")
make_schedule()

In [ ]:
from datasets import Dataset
import json
dataset = Dataset.from_json('/content/output.json')

In [ ]:
dataset["text"]

In [ ]:
from datasets import Dataset
import json

dataset_finetune = dataset.map(formating_prompt_func)
dataset_finetune["text"]

In [ ]:
!pip install --upgrade cut_cross_entropy triton

In [ ]:
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade cut_cross_entropy triton

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_finetune,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    dataset_num_proc = 2,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="text",
        report_to="none",
    ),
)

In [ ]:
trainer_start = trainer.train()

In [ ]:
input = "what is your name"
FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(input,"","")],return_tensors="pt").to("cuda")
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=500,
    use_cache=True,
)

In [ ]:
Response = tokenizer.batch_decode(outputs)
print(Response[0].split("##Response:")[1])

In [ ]:
import re

# --- Function Mapping ---
def execute_function_call(model_output):
    # Extract function name
    # Join the list of strings into a single string
    model_output_str = " ".join(model_output)
    func_name = re.search(r"<function>([^<]+)</function>", model_output)
    print(func_name.group(1).strip())
    if not func_name:
        return model_output  # No function detected

    func_name = func_name.group(1).strip()

    # Call the function (no args)
    if func_name == 'make_schedule':
        result = make_schedule()
    else:
        return f"Error: Function '{func_name}' not found."

    return f"{func_name} result: {result}"

# --- Test the Model ---
model_output = Response[0].split("##Response:")[1]

# Execute the function
final_output = execute_function_call(model_output)
print(final_output)

In [ ]:
import re
from datetime import datetime

def execute_function_call(model_output):
    # Extract day and time ranges
    day_time_ranges = re.findall(r'([A-Za-z]+)\s(\d{1,2}:\d{2}-\d{1,2}:\d{2})', model_output)

    if not day_time_ranges:
        print(model_output)  # No day and time ranges detected

    current_day = datetime.now().strftime("%A")
    current_time = datetime.now().time()

    print("Extracted Day & Time Ranges:", day_time_ranges)
    print("Current Day:", current_day)
    print("Current Time:", current_time)

    # Function to check if current time is within a given time range
    def is_time_in_range(time_range, current_time):
        start_str, end_str = time_range.split('-')
        start = datetime.strptime(start_str, "%H:%M").time()
        end = datetime.strptime(end_str, "%H:%M").time()
        return start <= current_time <= end

    # Check each extracted day and time range
    for day, time_range in day_time_ranges:
        if day.lower() == current_day.lower():
            print(f"\nToday is {current_day}, checking time range: {time_range}")
            if is_time_in_range(time_range, current_time):
                print("✅ lecturer is available")
            else:
                print("❌ lecturer is unavailable at the moment")
        else:
            print(f"\nToday is {current_day}, skipping {day}'s range: {time_range}")
model_output = Response[0].split("##Response:")[1]

final_output = execute_function_call(model_output)
print(final_output)